## FineTuning Stable-Diffusion  
Fine Tuning Stable Diffusion on Any Data.
We recommended using multi-GPU. But if you have just only 1 GPU, VRAM will be at least 24GB(1 RTX 3090 will not be enough, so do I). It can run on Colab Pro or Pro+

In [ ]:
!pip install huggingface_hub
!git clone https://github.com/42brick/Lego-Diffusion.git
!mkdir -p ./Lego-Diffusion/models/ldm/stable-diffusion-v1

%cd Lego-diffusion
!pip install --upgrade pip
!pip install -r requirements.txt

# if colab, torchtext module might get error. run these code
!pip install --upgrade keras # on lambda stack we need to upgrade keras
!pip uninstall -y torchtext # on colab we need to remove torchtext

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# Check the dataset
'''
1. You need to upload your dataset in huggingface with .parquet file
    (you can make parquet file using open_parquet.ipynb)
    
2. You need to make dataset_infos.json
    (I dont know how to make it exactly, but i'm handwritting that)
    
3. load_dataset with under code

* "HoneyTylenol/lego_face_final" data is private data. 
'''
from datasets import load_dataset
ds = load_dataset("HoneyTylenol/lego_face_final", split="train")
sample = ds[0]
display(sample["image"].resize((256, 256)))
print(sample["text"])

In [ ]:
from huggingface_hub import hf_hub_download
ckpt_path = hf_hub_download(repo_id="CompVis/stable-diffusion-v-1-4-original", filename="sd-v1-4-full-ema.ckpt", use

In [ ]:
# A100
BATCH_SIZE = 2
N_GPUS = 1

gpu_list = ",".join((str(x) for x in range(N_GPUS))) + ","
print(f"Using GPUs: {gpu_list}")

In [ ]:
# Run training
!(python main.py \
    -t \
    --base configs/stable-diffusion/lego.yaml \
    --gpus "$gpu_list" \
    --scale_lr False \
    --num_nodes 1 \
    --check_val_every_n_epoch 10 \
    --finetune_from "$ckpt_path" \
    data.params.batch_size="$BATCH_SIZE" \
)

In [ ]:
# Run the model
!(python scripts/txt2img.py \
    --prompt 'robotic cat with wings' \
    --outdir 'outputs/generated_pokemon' \
    --H 512 --W 512 \
    --n_samples 4 \
    --config 'configs/stable-diffusion/pokemon.yaml' \
    --ckpt 'path/to/your/checkpoint')